In [1]:
%load_ext autoreload
%autoreload 2
from airo_drake.stations.single_ur3e import MakeUR3eCartStation, RunAndPublishSimulation
from airo_drake.stations.single_ur3e_real import UR3eStationHardwareInterface, RunAndPublishOnRealHardware

from pydrake.geometry import StartMeshcat

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [3]:
import numpy as np
from pydrake.all import AngleAxis, PiecewisePose, Rgba, RigidTransform, RotationMatrix

from airo_drake.cloth_manipulation.towel import fake_towel_keypoints, order_keypoints
from airo_drake.geometry import top_down_orientation
from airo_drake.visualization import VisualizePath
from airo_drake.planners.planners_base import PlannerBase


class DemoPlanner(PlannerBase):
    def __init__(self, plant, meshcat):
        super().__init__(plant, meshcat)
        self.towel_keypoints = fake_towel_keypoints(height=0.0)  + np.array([0.45, 0, 0]).reshape((3,1))
        VisualizePath(meshcat, "towel_keypoints", self.towel_keypoints, closed=True, color=Rgba(0, 1, 1), thickness=10)

    def get_fold_keyposes(self, keypoints):
        left_grasp_pose = None
        ordered_keypoints = order_keypoints(keypoints)

        top_right_corner, top_left_corner, bottom_left_corner, bottom_right_corner = ordered_keypoints.T

        right_edge = top_right_corner - bottom_right_corner

        topdown = RotationMatrix(top_down_orientation(right_edge))
        tilt_angle = 15
        gripper_y = topdown.col(1)
        local_y_rotation = RotationMatrix(AngleAxis(np.deg2rad(tilt_angle), gripper_y))
        grasp_orientation = local_y_rotation @ topdown

        height_offset = np.array([0, 0, 0.01])
        grasp_position_left = bottom_left_corner + height_offset
        left_grasp_pose = RigidTransform(grasp_orientation, grasp_position_left)

        edge_direction = right_edge / np.linalg.norm(right_edge)
        pregrasp_approach_distance = 0.05
        approach_vector = -pregrasp_approach_distance * edge_direction
        approach_vector[2] += 0.05

        left_pregrasp_pose = RigidTransform(left_grasp_pose)
        left_pregrasp_pose.set_translation(left_pregrasp_pose.translation() + approach_vector)

        local_y_rotation = RotationMatrix(AngleAxis(np.deg2rad(-tilt_angle), gripper_y))
        release_orientation = local_y_rotation @ topdown
        left_release_position = top_left_corner + height_offset
        left_release_pose = RigidTransform(release_orientation, left_release_position)

        fold_height_offset = np.array([0, 0, np.linalg.norm(right_edge) / 2.0])
        left_middle_position = (grasp_position_left + left_release_position) / 2 + fold_height_offset

        left_middle_pose = RigidTransform(topdown, left_middle_position)

        left_retreat_position = left_release_position + np.array([0, 0, 0.1])
        left_retreat_pose = RigidTransform(topdown, left_retreat_position)

        keyposes = {
            "pregrasp": left_pregrasp_pose,
            "grasp": left_grasp_pose,
            "middle": left_middle_pose,
            "release": left_release_pose,
            "retreat": left_retreat_pose,
        }

        return keyposes



    def Plan(self, context, state):
        keyposes = self.get_fold_keyposes(self.towel_keypoints)
        self.tcp_keyposes = {"initial": self.inital_tcp, **keyposes}
        times = {"initial": 0.0, "pregrasp": 2.0, "grasp": 2.5, "middle": 3.25, "release": 4.0, "retreat": 5.0}

        def FoldingTrajectory(keyposes_dict, times_dict):
            keyposes = list(keyposes_dict.values())
            times = list(times_dict.values())
            pose_trajectory = PiecewisePose.MakeCubicLinearWithEndLinearVelocity(times, keyposes)
            return pose_trajectory


        self.tcp_trajectory = FoldingTrajectory(self.tcp_keyposes, times)

        self.UpdatePlanVisualization()



In [6]:
# station = MakeUR3eCartStation()
# plant = station.GetSubsystemByName("plant")
# planner = DemoPlanner(plant, meshcat) # Change this to your planner to test.
# RunAndPublishSimulation(station, planner, meshcat)



    directives:
    - add_directives:
        file: package://airo_drake_models/ur3e.dmd.yaml
    


In [5]:
station = UR3eStationHardwareInterface()
planner = DemoPlanner(None, meshcat) # Change this to your planner to test.
RunAndPublishOnRealHardware(station, planner, meshcat)

RECEIVED POSE
[0.1379993668661703, -0.16094517286761728, 0.1699000936288906, -3.1055637208046423, -0.005684486269960845, -0.029064676179565402]
[[ 0.9998182   0.00399511  0.01864441  0.13799937]
 [ 0.00332355 -0.99934941  0.03591264 -0.16094517]
 [ 0.01877576 -0.03584415 -0.999181    0.16990009]
 [ 0.          0.          0.          1.        ]]
RECEIVED POSE
[0.1379815008420165, -0.16095182471444844, 0.16989540906334852, -3.1055890767289895, -0.005667264587537666, -0.029011356054670902]
[[ 0.99981888  0.00398315  0.01861019  0.1379815 ]
 [ 0.00331328 -0.99935034  0.03588767 -0.16095182]
 [ 0.01874105 -0.03581951 -0.99918253  0.16989541]
 [ 0.          0.          0.          1.        ]]
RECEIVED POSE
[0.13799215256137767, -0.1609527040785546, 0.1699005897286976, -3.1055883324360427, -0.005611273158944971, -0.02904628059568485]
[[ 0.99981859  0.00394751  0.01863332  0.13799215]
 [ 0.00327683 -0.99935045  0.03588789 -0.1609527 ]
 [ 0.01876289 -0.03582032 -0.99918209  0.16990059]
 [ 0.